In [196]:
import pandas as pd

### Reading and veryfing data

In [197]:
# read csv file into dataframe
matches = pd.read_csv('matches.csv', index_col=0)

matches.shape

(1520, 27)

We check the shape of our dataframe and we find that we have 1520 rows. This means that we no missing match data. This is because in the EPL teams play 38 games, and there are 20 teams. So we have 38 * 20 * 2 = 1520. One thing to note is that in the EPL every year the bottom 3 teams get 'relegated' meaning that they move down into the lower league called the English Championship. Also, the top 3 teams from the Englsih Championship get 'promoted' to the EPL. So we will see that we actually have more than 20 teams and that is because certain teams did not participate in both years. We can verify this by showing the amount of values based on each team name.

In [198]:
matches['team'].value_counts()

Manchester City             76
Wolverhampton Wanderers     76
Burnley                     76
Leeds United                76
Everton                     76
Southampton                 76
Aston Villa                 76
Liverpool                   76
Newcastle United            76
Crystal Palace              76
Brighton and Hove Albion    76
Leicester City              76
West Ham United             76
Manchester United           76
Arsenal                     76
Tottenham Hotspur           76
Chelsea                     76
Brentford                   38
Watford                     38
Norwich City                38
Fulham                      38
West Bromwich Albion        38
Sheffield United            38
Name: team, dtype: int64

As we can see above all teams with 76 games competed in both seasons. The teams with 38 games were either promoted or releagted and only competed in one of the two seasons in our dataset.

### Cleaning our Data For Machine Learning
For machine learning in order to use columns in our predictions they must be int for float types. There are a few columns that we are going to want to convert to different types. Date is currently and object, but we want convert it to datetime as it allows us to easily perform calculations. Next the `venue` column indicated wether the match was played at home or away. In [the EPL playing at home gives teams a large advantage](https://bleacherreport.com/articles/1604854-how-much-does-home-field-advantage-matter-in-soccer) vs playing away. Currently that column is an object, but we want to convert it to a numerical value so it can be used in our analysis. We also are interested if the time or day has any effect on the outcome so we need to create a new columns that have this data in numerical form.

In [199]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf                int64
ga                int64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

First lets convert the date column into a datetime column

In [200]:
# convert date column to type datetime
matches['date'] = pd.to_datetime(matches['date'])

# add venue_code that corresponds to home or away
matches['venue_code'] = matches['venue'].astype('category').cat.codes

# add venue code correponding to opponent
matches['opp_code'] = matches['venue'].astype('category').cat.codes

# create new column hour extracted from date
matches['hour'] = matches['time'].str.replace(':.+', '', regex=True).astype('int')

# create day_code column that has a code the corresponds with day of the week
matches['day_code'] = matches['date'].dt.dayofweek

# create new column target that stores result of match 0 of team lost or drew, 1 if team 1
matches['target'] = (matches['result'] == 'W').astype('int')

In [201]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,1.0,0.0,0.0,2022,Manchester City,0,0,16,6,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,1.0,0.0,0.0,2022,Manchester City,1,1,15,5,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,0.0,0.0,0.0,2022,Manchester City,1,1,12,5,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,0.0,0.0,0.0,2022,Manchester City,0,0,15,5,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,1.0,0.0,0.0,2022,Manchester City,1,1,15,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,0.0,0.0,0.0,2021,Sheffield United,0,0,19,6,0
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,1.0,0.0,0.0,2021,Sheffield United,1,1,15,5,0
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,0.0,0.0,0.0,2021,Sheffield United,0,0,19,6,1
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,1.0,0.0,0.0,2021,Sheffield United,0,0,18,2,0


# Training Machine Learning Model


In [202]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

train = matches[matches['date'] < '2022-01-01']
test = matches[matches['date'] >= '2022-01-01']

predictors = ['venue_code', 'opp_code', 'hour', 'day_code']

rf.fit(train[predictors], train['target'])

preds = rf.predict(test[predictors])

In [203]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(test['target'], preds)
acc

0.5939086294416244

In [204]:
combined = pd.DataFrame(dict(actual=test['target'], predict=preds))

pd.crosstab(index=combined['actual'], columns=combined['predict'])

predict,0,1
actual,,
0,229,7
1,153,5


In [205]:
from sklearn.metrics import precision_score

precision_score(test['target'], preds)

0.4166666666666667

So we can see when predicting a win we only had 42% accuracy. We are going to try to improve our accuracy using rolling averages for each teams performance based on recent form. To do this we are going to use grouped matches

# Improving Model Accuracy using Rolling Averages

In [206]:
grouped_matches = matches.groupby('team')
group = grouped_matches.get_group('Manchester City')
group

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,venue_code,opp_code,hour,day_code,target
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,1.0,0.0,0.0,2022,Manchester City,0,0,16,6,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,1.0,0.0,0.0,2022,Manchester City,1,1,15,5,1
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,0.0,0.0,0.0,2022,Manchester City,1,1,12,5,1
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,0.0,0.0,0.0,2022,Manchester City,0,0,15,5,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,1.0,0.0,0.0,2022,Manchester City,1,1,15,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,2021-05-01,12:30,Premier League,Matchweek 34,Sat,Away,W,2,0,Crystal Palace,...,1.0,0.0,0.0,2021,Manchester City,0,0,12,5,1
56,2021-05-08,17:30,Premier League,Matchweek 35,Sat,Home,L,1,2,Chelsea,...,0.0,0.0,1.0,2021,Manchester City,1,1,17,5,0
57,2021-05-14,20:00,Premier League,Matchweek 36,Fri,Away,W,4,3,Newcastle Utd,...,1.0,0.0,0.0,2021,Manchester City,0,0,20,4,1
58,2021-05-18,19:00,Premier League,Matchweek 37,Tue,Away,L,2,3,Brighton,...,1.0,0.0,0.0,2021,Manchester City,0,0,19,1,0


In [207]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group
    

In [208]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f"{c}_rolling" for c in cols]
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [209]:
matches_rolling = matches.groupby('team').apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2,1,Sheffield Utd,...,6,1,2.000000,1.333333,8.000000,3.666667,14.633333,0.666667,0.000000,0.000000
7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0,1,Manchester City,...,5,0,1.666667,1.666667,5.666667,3.666667,15.366667,0.000000,0.000000,0.000000
9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0,1,Leicester City,...,6,0,1.000000,1.666667,7.000000,3.666667,16.566667,0.666667,0.000000,0.000000
11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1,0,Manchester Utd,...,6,1,0.666667,1.000000,9.666667,4.000000,16.566667,1.000000,0.000000,0.000000
13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0,3,Aston Villa,...,6,0,0.333333,0.666667,9.666667,2.666667,19.333333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Home,L,0,3,Brighton,...,5,0,0.666667,1.000000,8.666667,3.333333,17.400000,0.000000,0.000000,0.000000
38,2022-05-07,15:00,Premier League,Matchweek 36,Sat,Away,D,2,2,Chelsea,...,5,0,0.000000,1.666667,8.666667,2.333333,18.666667,0.333333,0.000000,0.000000
39,2022-05-11,20:15,Premier League,Matchweek 33,Wed,Home,L,1,5,Manchester City,...,2,0,0.666667,2.000000,11.666667,3.000000,17.800000,0.333333,0.000000,0.000000
40,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Home,D,1,1,Norwich City,...,6,0,1.000000,3.333333,10.666667,2.666667,17.100000,0.333333,0.000000,0.000000


In [210]:
matches_rolling.index = range(matches_rolling.shape[0])
matches_rolling

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2,1,Sheffield Utd,...,6,1,2.000000,1.333333,8.000000,3.666667,14.633333,0.666667,0.000000,0.000000
1,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0,1,Manchester City,...,5,0,1.666667,1.666667,5.666667,3.666667,15.366667,0.000000,0.000000,0.000000
2,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0,1,Leicester City,...,6,0,1.000000,1.666667,7.000000,3.666667,16.566667,0.666667,0.000000,0.000000
3,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1,0,Manchester Utd,...,6,1,0.666667,1.000000,9.666667,4.000000,16.566667,1.000000,0.000000,0.000000
4,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0,3,Aston Villa,...,6,0,0.333333,0.666667,9.666667,2.666667,19.333333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Home,L,0,3,Brighton,...,5,0,0.666667,1.000000,8.666667,3.333333,17.400000,0.000000,0.000000,0.000000
1444,2022-05-07,15:00,Premier League,Matchweek 36,Sat,Away,D,2,2,Chelsea,...,5,0,0.000000,1.666667,8.666667,2.333333,18.666667,0.333333,0.000000,0.000000
1445,2022-05-11,20:15,Premier League,Matchweek 33,Wed,Home,L,1,5,Manchester City,...,2,0,0.666667,2.000000,11.666667,3.000000,17.800000,0.333333,0.000000,0.000000
1446,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Home,D,1,1,Norwich City,...,6,0,1.000000,3.333333,10.666667,2.666667,17.100000,0.333333,0.000000,0.000000


In [211]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [212]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [213]:
error

0.5294117647058824

We improved our accuracy of predicting a win to 53%

In [214]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)
combined.head(10)

,actual,predicted,date,team,opponent,result
55,0,0,2022-01-23,Arsenal,Burnley,D
56,1,0,2022-02-10,Arsenal,Wolves,W
57,1,0,2022-02-19,Arsenal,Brentford,W
58,1,1,2022-02-24,Arsenal,Wolves,W
59,1,1,2022-03-06,Arsenal,Watford,W
60,1,1,2022-03-13,Arsenal,Leicester City,W
61,0,1,2022-03-16,Arsenal,Liverpool,L
62,1,0,2022-03-19,Arsenal,Aston Villa,W
63,0,0,2022-04-04,Arsenal,Crystal Palace,L
64,0,0,2022-04-09,Arsenal,Brighton,L


In [215]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [216]:
combined["new_team"] = combined["team"].map(mapping)

In [217]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [218]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,0,0,2022-01-23,Arsenal,Burnley,D,Arsenal,0,0,Burnley,Arsenal,D,Burnley
1,1,0,2022-02-10,Arsenal,Wolves,W,Arsenal,0,1,Wolverhampton Wanderers,Arsenal,L,Wolves
2,1,0,2022-02-19,Arsenal,Brentford,W,Arsenal,0,0,Brentford,Arsenal,L,Brentford
3,1,1,2022-02-24,Arsenal,Wolves,W,Arsenal,0,0,Wolverhampton Wanderers,Arsenal,L,Wolves
4,1,1,2022-03-06,Arsenal,Watford,W,Arsenal,0,0,Watford,Arsenal,L,Watford
...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,0,0,2022-04-30,Wolverhampton Wanderers,Brighton,L,Wolves,1,0,Brighton and Hove Albion,Wolves,W,Brighton
384,0,1,2022-05-07,Wolverhampton Wanderers,Chelsea,D,Wolves,0,1,Chelsea,Wolves,D,Chelsea
385,0,0,2022-05-11,Wolverhampton Wanderers,Manchester City,L,Wolves,1,1,Manchester City,Wolves,W,Manchester City
386,0,0,2022-05-15,Wolverhampton Wanderers,Norwich City,D,Wolves,0,0,Norwich City,Wolves,D,Norwich City
